# Emerging Technologies – Problems

**Author:** Hammad Mubarik  
**Module:** Emerging Technologies  
**Year:** 2026

## Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#deutschs-algorithm) is a quantum algorithm that figures out whether a given function is constant or balanced in one query. A classical computer would need to check up to half the inputs before it could be sure.

To test the algorithm we need to be able to generate these functions ourselves. The goal here is to write a function called `random_constant_balanced` that returns a random function taking four Boolean inputs and returning a single Boolean output, where the returned function is guaranteed to be either constant or balanced.

- **Constant** means the function returns the same value no matter what you pass in.
- **Balanced** means it returns `True` for exactly half the possible inputs and `False` for the other half.

In [80]:
# Random selections.
import random

# Numerical arrays and operations.
import numpy as np

# Generating all combinations of inputs.
import itertools as it

### Representing Boolean Functions as Lookup Tables

With four Boolean inputs there are $2^4 = 16$ possible input combinations. Rather than writing out the function logic explicitly, I'm representing each function as a lookup table — a tuple of 16 output values where the position corresponds to the input.

To use it, the four inputs $(a, b, c, d)$ get converted to an integer index and we just read off the value at that position. So for example $(False, True, False, True)$ becomes `0101` in binary which is 5, so we look at index 5 in the table.

This makes it straightforward to control whether the function is constant or balanced when building the table, and the table gets captured in a [closure](https://realpython.com/python-closures/) so the returned function carries it around internally.

In [81]:
# Show all 16 possible input combinations for four Boolean inputs.
# itertools.product generates the cartesian product of the input iterables.
# See: https://docs.python.org/3/library/itertools.html#itertools.product
all_inputs = list(it.product([False, True], repeat=4))

# Show the first few and the total count.
print(f'Total input combinations: {len(all_inputs)}')
print('First four inputs:', all_inputs[:4])
print('Last  four inputs:', all_inputs[-4:])

Total input combinations: 16
First four inputs: [(False, False, False, False), (False, False, False, True), (False, False, True, False), (False, False, True, True)]
Last  four inputs: [(True, True, False, False), (True, True, False, True), (True, True, True, False), (True, True, True, True)]


### Converting Boolean Inputs to an Index

Each of the four inputs is treated as a bit — `True` is `1` and `False` is `0`. Putting them together gives a 4-bit binary number which we convert to a decimal integer to use as the index.

So $(True, False, True, True)$ becomes $1011_2 = 8 + 0 + 2 + 1 = 11$.

Python's built-in [`int`](https://docs.python.org/3/library/functions.html#int) function handles the binary string to integer conversion directly with base 2.

Here is a quick reference table showing decimal 0–20 and their binary equivalents, which is useful for understanding how the inputs map to positions in the lookup table.

In [82]:
# Display a decimal to binary reference table for 0 to 20.
# bin() returns a string like '0b1011'; [2:] strips the '0b' prefix.
# See: https://docs.python.org/3/library/functions.html#bin
print('Below is a reference table for decimal to binary conversion')
print(f'{"Decimal":>10} | {"Binary":>8}')
print('-' * 22)
for n in range(21):
    print(f'{n:>10} | {bin(n)[2:]:>8}')

Below is a reference table for decimal to binary conversion
   Decimal |   Binary
----------------------
         0 |        0
         1 |        1
         2 |       10
         3 |       11
         4 |      100
         5 |      101
         6 |      110
         7 |      111
         8 |     1000
         9 |     1001
        10 |     1010
        11 |     1011
        12 |     1100
        13 |     1101
        14 |     1110
        15 |     1111
        16 |    10000
        17 |    10001
        18 |    10010
        19 |    10011
        20 |    10100


In [83]:
def bool_args_to_index(a, b, c, d):
    """Convert four Boolean arguments to an integer index.
    
    Each argument is treated as a bit (1 if true, 0 if false).
    The four bits are joined into a binary string and converted to an int.
    
    Example: (True, False, True, True) -> '1011' -> 11 (See table above)
    """
    # Map each argument to '1' or '0'.
    bits = ''.join('1' if x else '0' for x in (a, b, c, d))
    # Convert the binary string to an integer using base 2.
    return int(bits, 2)

In [84]:
# Quick check: verify the index for a known input.
# (True, False, True, True) is binary 1011, which equals 11. (See table above)
print(bool_args_to_index(True, False, True, True))

if(bool_args_to_index(True, False, True, True) == 11):
    print('Test passed: (True, False, True, True) correctly maps to index 11')

# (False, False, False, False) is binary 0000, which equals 0. (See table above)
print(bool_args_to_index(False, False, False, False))

if(bool_args_to_index(False, False, False, False) == 0):
    print('Test passed: (False, False, False, False) correctly maps to index 0')

# (True, True, True, True) is binary 1111, which equals 15. (See table above)
print(bool_args_to_index(True, True, True, True))


if(bool_args_to_index(True, True, True, True) == 15):
    print('Test passed: (True, True, True, True) correctly maps to index 15')


11
Test passed: (True, False, True, True) correctly maps to index 11
0
Test passed: (False, False, False, False) correctly maps to index 0
15
Test passed: (True, True, True, True) correctly maps to index 15


### Building the Lookup Table

The key step is constructing the 16-value lookup table:

- **Constant:** All 16 entries are the same value — either all `False` or all `True`. There are exactly 2 constant functions.
- **Balanced:** Exactly 8 entries are `True` and 8 are `False`, arranged in a random order. The number of such functions is $\binom{16}{8} = 12{,}870$.

We first randomly choose the type (50/50), then build the table accordingly. [`random.shuffle`](https://docs.python.org/3/library/random.html#random.shuffle) is used to randomise the placement of `True` values in the balanced case.

In [85]:
def random_constant_balanced():
    """Return a randomly chosen constant or balanced function of four Boolean inputs.

    The returned function accepts exactly four Boolean arguments and returns
    a single Boolean value. It is guaranteed to be either:
      - Constant: returns the same value for all 16 possible inputs, or
      - Balanced: returns True for exactly 8 of the 16 possible inputs.

    The function type (constant or balanced) is chosen with equal probability.
    Within each type, the specific function is also chosen uniformly at random.
    """
    # There are 2^4 = 16 possible inputs for four Boolean arguments.
    num_inputs = 2 ** 4

    # Randomly choose whether the function is constant or balanced.
    is_constant = random.choice([True, False])

    if is_constant:
        # Constant: every entry in the lookup table is the same value.
        # Choose randomly between always-False and always-True.
        value = random.choice([False, True])
        lookup = (value,) * num_inputs
    else:
        # Balanced: exactly half the entries are True, half are False.
        # Start with 8 False values followed by 8 True values.
        lookup = [False] * (num_inputs // 2) + [True] * (num_inputs // 2)
        # Shuffle so the True values appear in random positions.
        random.shuffle(lookup)
        # Convert to a tuple so the table is immutable.
        lookup = tuple(lookup)

    def f(a, b, c, d):
        """Take four Boolean inputs and return a Boolean output."""
        # Convert the inputs to an index into the lookup table.
        idx = bool_args_to_index(a, b, c, d)
        # Return the value stored at that index.
        return lookup[idx]

    # Return the closure — the lookup table is captured inside f.
    return f

### Demonstrating the Function

We can verify that `random_constant_balanced` works correctly by calling the returned function on all 16 possible inputs and inspecting the outputs. A helper function, `classify`, checks whether the outputs are constant or balanced.

In [86]:
def classify(f):
    """Classify a four-input Boolean function as constant or balanced.

    Calls f on all 16 possible inputs and checks the outputs.
    Returns 'constant' if all outputs are the same, 'balanced' if
    exactly half are True, or 'other' if neither condition holds.
    """
    # Evaluate f on every possible four-bit input.
    outputs = [f(a, b, c, d) for a, b, c, d in it.product([False, True], repeat=4)]

    # Count how many outputs are True.
    true_count = sum(outputs)

    if true_count == 0 or true_count == 16:
        return 'constant'
    elif true_count == 8:
        return 'balanced'
    else:
        return 'other'

In [87]:
# Generate and classify five random functions to show the variety.
for i in range(5):
    f = random_constant_balanced()
    outputs = [f(a, b, c, d) for a, b, c, d in it.product([False, True], repeat=4)]
    kind = classify(f)
    print(f'Function {i + 1}: {kind:9s} | outputs: {[int(v) for v in outputs]}')

Function 1: constant  | outputs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Function 2: constant  | outputs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Function 3: balanced  | outputs: [1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1]
Function 4: balanced  | outputs: [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1]
Function 5: balanced  | outputs: [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]


### Verifying Correctness at Scale

To gain confidence that `random_constant_balanced` never produces an invalid function, we generate a large number of functions and confirm that every single one is classified as either constant or balanced — never anything else.

In [88]:
# Generate 10,000 random functions and record their classifications.
num_trials = 10_000
results = [classify(random_constant_balanced()) for _ in range(num_trials)]

# Count each classification.
counts = {label: results.count(label) for label in ['constant', 'balanced', 'other']}

print(f'Trials:   {num_trials}')
print(f'Constant: {counts["constant"]}')
print(f'Balanced: {counts["balanced"]}')
print(f'Other:    {counts["other"]}  <- must be 0')

# Assert that no invalid functions were generated.
assert counts['other'] == 0, 'Invalid function detected!'
print('\nAll functions are valid (constant or balanced).')

Trials:   10000
Constant: 4884
Balanced: 5116
Other:    0  <- must be 0

All functions are valid (constant or balanced).


## Problem 2: Classical Testing for Function Type

Deutsch's algorithm gets the answer in a single query to the function. Before we look at the quantum side of that, it helps to understand what the classical cost actually is  how many times do you need to call the function before you can be completely certain whether it is constant or balanced?

The task here is to write a function called `determine_constant_balanced` that takes a function `f` (as returned by `random_constant_balanced` from Problem 1) and returns the string `"constant"` or `"balanced"` depending on what `f` is.

### Strategy

With four Boolean inputs there are $2^4 = 16$ possible inputs. A balanced function has **exactly 8 True and 8 False** outputs. That constraint is the key:

- If we ever see **two different outputs**, the function must be balanced a constant function cannot do that.
- If we have seen the **same output 9 times in a row**, the function must be constant — a balanced function only has 8 of any one value, so it cannot produce the same output 9 times.

This means we can stop checking as soon as one of those two conditions is met rather than evaluating all 16 inputs.

In [89]:
def determine_constant_balanced(f):
    """Determine whether f is constant or balanced.

    Calls f one input at a time and stops as soon as the result is certain.
    A balanced function has exactly 8 True outputs out of 16, so seeing the
    same output 9 times guarantees the function is constant.

    Returns 'constant' or 'balanced'.
    Maximum calls to f: 9  (2^(n-1) + 1 where n = 4).
    """
    # All 16 possible inputs for four Boolean arguments.
    all_inputs = list(it.product([False, True], repeat=4))

    # First call establishes a baseline output to compare against.
    first_output = f(*all_inputs[0])

    # Check up to 8 more inputs (9 total).
    # If a different output is seen, the function must be balanced.
    # If all 9 match, the function must be constant.
    for inp in all_inputs[1:9]:
        if f(*inp) != first_output:
            return 'balanced'

    return 'constant'

### Efficiency

The maximum number of calls needed to be 100% certain is **9**.

In the worst case the function is constant, and every output matches the first so we have to check all 9 before we can rule out balanced. If the function happens to be balanced, we get lucky and stop as soon as we see a differing output, which can happen as early as the second call.

For a general $n$-bit input function, the worst-case number of classical queries is $2^{n-1} + 1$. With $n = 4$ that gives $8 + 1 = 9$. The [Deutsch–Jozsa algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#the-deutsch-jozsa-algorithm) solves the same problem in exactly **1 query**, regardless of $n$ that is the quantum advantage.

### Testing the Function

First, I test `determine_constant_balanced` against two functions I know the answer to  one constant and one balanced  just to confirm it returns the right string before running it on random inputs.

In [90]:
# A known constant function — always returns False.
always_false = lambda a, b, c, d: False

# A known balanced function  returns True if and only if a is True.
# Exactly 8 of the 16 inputs have a=True, so this is balanced.
half_true = lambda a, b, c, d: a

# Check both.
print(determine_constant_balanced(always_false))
print(determine_constant_balanced(half_true))

constant
balanced


Now I run it against 10,000 random functions and check that `determine_constant_balanced` always agrees with `classify`. Since `classify` checks all 16 outputs it is guaranteed to be correct, so any disagreement would mean a bug in `determine_constant_balanced`.

In [ ]:
# Run 10,000 trials and compare determine_constant_balanced against classify.
num_trials = 10_000
mismatches = 0

for _ in range(num_trials):
    f = random_constant_balanced()
    # classify checks all 16 outputs — guaranteed correct.
    expected = classify(f)
    # determine_constant_balanced uses at most 9 calls.
    result = determine_constant_balanced(f)
    if result != expected:
        mismatches += 1

print(f'Trials:     {num_trials}')
print(f'Mismatches: {mismatches}  <- must be 0')

assert mismatches == 0, 'determine_constant_balanced gave a wrong answer!'
print('\ndetermine_constant_balanced is correct on all trials.')

## Problem 3: Quantum Oracles

[Deutsch's algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#deutschs-algorithm) is the simplest example of a quantum algorithm that uses superposition to determine a global property of a function in a single evaluation — something that is impossible classically without at least two calls.

For a single Boolean input there are exactly four possible functions, as covered in Problem 1: two constant (always 0, always 1) and two balanced (identity, NOT). In the quantum setting each of these functions must be encoded as a **quantum oracle** before Deutsch's algorithm can use it.

A quantum oracle implements the transformation

$$|x\rangle|y\rangle \rightarrow |x\rangle|y \oplus f(x)\rangle$$

where $\oplus$ is XOR. The input qubit $|x\rangle$ is left unchanged and the result of $f(x)$ is XORed into an auxiliary qubit $|y\rangle$. This keeps the operation reversible, which is a requirement for all quantum gates.

In [ ]:
# Quantum circuit construction.
import qiskit

# Quantum simulator backend.
import qiskit_aer as aer

### The Four Single-Bit Functions

As established in Problem 1, there are four functions that take a single Boolean input and return a single Boolean output.

| Function | f(0) | f(1) | Type |
|----------|------|------|------|
| Constant 0 | 0 | 0 | Constant |
| Constant 1 | 1 | 1 | Constant |
| Identity | 0 | 1 | Balanced |
| NOT | 1 | 0 | Balanced |

Each of these needs its own quantum oracle. The oracle is a two-qubit circuit — one input qubit $|x\rangle$ and one auxiliary qubit $|y\rangle$ — and is built using combinations of the X gate and the CX (controlled-NOT) gate.